實作向量搜索_01

In [ ]:
# getpass：安全地提示用戶輸入密碼或其他敏感信息
import getpass, os, pymongo, pprint
# PyPDFLoader：加載 PDF 文件並將其內容轉換為可處理的文本或數據結構
from langchain_community.document_loaders import PyPDFLoader
# StrOutputParser：將模型生成的輸出解析為字串
from langchain_core.output_parsers import StrOutputParser
# RunnablePassthrough：將輸入直接傳遞到下一步而不進行任何處理的可運行單元
from langchain_core.runnables import RunnablePassthrough
# 與 MongoDB Atlas Vector Search 集成，以實現向量搜索功能
from langchain_mongodb import MongoDBAtlasVectorSearch
# ChatOpenAI 用於與 OpenAI 的聊天模型交互
# OpenAIEmbeddings 用於生成文本的向量嵌入
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# 定義和管理提示模板，以生成特定的查詢或指令
from langchain.prompts import PromptTemplate
# 將長文本拆分為較小的片段，以便於處理和分析。
from langchain.text_splitter import RecursiveCharacterTextSplitter
# MongoDB
from pymongo import MongoClient
# SSL
import certifi
# Streamlit
import streamlit as st
#
import pprint

os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]
ATLAS_CONNECTION_STRING = st.secrets["MONGODB_URL"]

# 建立連線
client = MongoClient(ATLAS_CONNECTION_STRING, tlsCAFile=certifi.where())
# 定義資料庫與集合名稱
db_name = "MyDatabase2024"
collection_name = "MyCollection2024"
atlas_collection = client[db_name][collection_name]
vector_search_index = "vector_index"

# 載入 PDF
loader = PyPDFLoader("古典測量理論.pdf")
# loader = PyPDFLoader("證書及作品集_原始.pdf")
# loader = PyPDFLoader("論文01.pdf")

data = loader.load()
# 文件分割器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
# 分割文件
docs = text_splitter.split_documents(data)
# 可輸出觀察看看
print(docs[0], "\n")

# 建立向量儲存
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(disallowed_special=()),
    collection=atlas_collection,
    index_name=vector_search_index,
)

query = "MongoDB Atlas security"
results = vector_search.similarity_search(query)
pprint.pprint(results)

# 將 Atlas Vector Search 實例化為擷取器
retriever = vector_search.as_retriever(
    # 指定搜索類型為相似度搜索
    search_type="similarity",
    # 設定查詢時僅返回前 10 個相關性最高的文檔，並且只使用分數高於 0.75 的文檔
    search_kwargs={"k": 10, "score_threshold": 0.75},
)
# 定義提示模板
template = """
使用以下內容來回答最後的問題。
如果你不知道答案，就說你不知道，不要試圖編造答案。
{context}
問題：{question}
"""

# 根據模板建立一個提示對象
custom_rag_prompt = PromptTemplate.from_template(template)
# 建立一個 OpenAI 聊天模型
llm = ChatOpenAI()


# 定義格式化文檔的函數
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# 建立一條鏈來回答有關您的數據的問題
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

# 問題
question = "如何使用古典真分數理論？"
# question = "簡述這篇論文的研究方法"

# 獲取與問題相關的文檔
print("\n相關文檔：")
documents = retriever.get_relevant_documents(question)
# print("\nSource documents:")
pprint.pprint(documents)


print("\n問題：" + question)

# 回答
answer = rag_chain.invoke(question)
print("\n回答：" + answer)

_特別注意，一定要進入資料庫設定 Atlas Search_

僅提問：修正中

In [13]:

import os
import certifi
from pymongo import MongoClient
import streamlit as st
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
import pprint
from langchain.docstore.document import Document

# 配置 OpenAI API Key 和 MongoDB 连接字串
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]
ATLAS_CONNECTION_STRING = st.secrets["MONGODB_URL"]

# 建立 MongoDB 连接
client = MongoClient(ATLAS_CONNECTION_STRING, tlsCAFile=certifi.where())
db_name = "MyDatabase2024"
collection_name = "MyCollection2024"
atlas_collection = client[db_name][collection_name]
vector_search_index = "vector_index"

# 从 MongoDB 載入数据
documents = []
for doc in atlas_collection.find():
    if "page_content" in doc:
        documents.append(Document(page_content=doc["page_content"], metadata={}))

# 使用已有的嵌入向量进行相似度搜索
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(disallowed_special=()),
    collection=atlas_collection,
    index_name=vector_search_index,
)

# 定义 RAG（检索增强生成）链
retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10, "score_threshold": 0.75},
)

template = """
使用以下內容來回答最後的問題。
如果你不知道答案，就說你不知道，不要試圖編造答案。
{context}
問題：{question}
"""

custom_rag_prompt = PromptTemplate.from_template(template)
llm = ChatOpenAI()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

question = "古典測量與心理測量的關係？"
documents = retriever.get_relevant_documents(question)
#
pprint.pprint(documents)

try:
    formatted_docs = format_docs(documents)
    print(f"Formatted documents: {formatted_docs}")
    # 确保上下文是字串
    if isinstance(formatted_docs, str) and isinstance(question, str):
        answer = rag_chain.invoke(question)
        print("\n回答：" + answer)
    else:
        print("Error: Formatted documents or question is not a string.")
except Exception as e:
    print(f"Error during RAG chain invoke: {e}")


=== documents 內容如下 ===
[Document(page_content='5. 古典測驗理論假設得分相同的受試者，其能力必然相同，而事實上，得分相同的受\n試者，其能力估計值會因為反應組型之不同而可能不同。 \n2.3 現代測驗理論 \n現代測驗理論正是為了改進古典測驗理論的缺失而誕生 ，它具有下列幾項特點 ，這\n些特點正是古典測驗理論所無法具備的（ Hambleton & Swam inathan，1985[14];', metadata={'_id': {'$oid': '6654317ab771faa6b60b35c1'}, 'source': '古典測量理論.pdf', 'page': 1}),
 Document(page_content='5. 古典測驗理論假設得分相同的受試者，其能力必然相同，而事實上，得分相同的受\n試者，其能力估計值會因為反應組型之不同而可能不同。 \n2.3 現代測驗理論 \n現代測驗理論正是為了改進古典測驗理論的缺失而誕生 ，它具有下列幾項特點 ，這\n些特點正是古典測驗理論所無法具備的（ Hambleton & Swam inathan，1985[14];', metadata={'_id': {'$oid': '66545022b771faa6b60b362b'}, 'source': '古典測量理論.pdf', 'page': 1}),
 Document(page_content='80[12])。以下就古典測驗理論和現代測驗理論分述如下： \n2.2 古典測驗理論內容 \n古典測驗理論又稱傳統測驗理論( conventional test theory )，或古典真分數理論\n(class ical true score theory )。 \n古典真分數理論的假定主要在界定觀察分數 、真分數及誤差分數三者間的關係 。其\n主要者有三：', metadata={'_id': {'$oid': '6654317ab771faa6b60b35ba'}, 'source': '古典測量理論.pdf', 'page': 0}),
 Document(page_content='80[12])。以下就古典測驗理論和現代測驗理論分述如下： \n2.2 古典測驗理論內容 \n古典測驗理論又稱傳統測驗理論( co